# Libraries

In [1]:
import openai
import tenacity

# Setup

In [2]:
@tenacity.retry(
    stop=tenacity.stop_after_attempt(3),
    wait=tenacity.wait_exponential(multiplier=1, min=1, max=60),
    retry=tenacity.retry_if_exception_type(openai.OpenAIError),
    reraise=True,
)
def generate_completion(prompt):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.0,
    )
    completed_text = response.choices[0].message["content"]
    return completed_text

# Dataset

In [3]:
sentences = [
    "if it is snowing then set the temperature to 55 degrees",
    "switch off the lights if nobody is at home",
    "lock the doors when I leave home",
    "turn on the heater when it is below 50 degrees",
    "reduce the temperature by 5 degrees",
    "open the door if my dog wants to go out",
]

# Tests

## Basic

Just identify the `condition` and `action`:

In [4]:
for sentence in sentences:

    prompt = f"""
    You are a syntax parser for a home automation system. Given a user's sentence like:
    
    if <condition> then <action>
    
    perform the following:

    1. Extract <condition> and <action>. If no condition, set it to None.
    2. Rewrite the extracted <condition> and <action> so that any of the user's personal and possessive pronouns are replaced with "the user"
    

    Process this sentence using the above steps:

    {sentence}

    Return the result in this format:

    {{
     
      "condition": <the rewritten condition from step 2>
      "action": <the rewritten action from step 2>

    }}

    """

    completion = generate_completion(prompt)

    print(f"Input: {sentence}")
    print(f"Output: {completion}")
    print()

Input: if it is snowing then set the temperature to 55 degrees
Output: {
     
      "condition": "if the user is experiencing snowing then",
      "action": "set the temperature to 55 degrees for the user"

    }

Input: switch off the lights if nobody is at home
Output: {
     
      "condition": "if nobody is at home",
      "action": "switch off the lights"

    }

Input: lock the doors when I leave home
Output: {
     
      "condition": "the user leaves home",
      "action": "lock the doors"

    }

Input: turn on the heater when it is below 50 degrees
Output: {
     
      "condition": "the temperature is below 50 degrees",
      "action": "turn on the heater"

    }

Input: reduce the temperature by 5 degrees
Output: {
     
      "condition": None,
      "action": "reduce the temperature by 5 degrees"

    }

Input: open the door if my dog wants to go out
Output: {
     
      "condition": "the user's dog wants to go out",
      "action": "open the door"

    }



Let's also include a message to acknowledge the user's request:

In [5]:
for sentence in sentences:

    prompt = f"""
      You are a syntax parser for a home automation system. Given a user's sentence like:
      
      if <condition> then <action>
      
      perform the following:

      1. Extract <condition> and <action>. If no condition, set it to None.
      2. Rewrite the extracted <condition> and <action> so that any of the user's personal and possessive pronouns are replaced with "the user".
      3. Create a friendly acknowledgement text. Replace any possessive references to the user with the corresponding second person
        possessive pronoun. References to the home automation system are to be in the first person.
      

      Process this sentence using the above steps:

      {sentence}

      Return the result in this format:

      {{
      
        "condition": <the rewritten condition from step 2>
        "action": <the rewritten action from step 2>
        "acknowledgement": <the acknowledgement text from step 3>

      }}
  """

    completion = generate_completion(prompt)

    print(f"Input: {sentence}")
    print(f"Output: {completion}")
    print()

Input: if it is snowing then set the temperature to 55 degrees
Output: {
  "condition": "if the user is experiencing snowing then set",
  "action": "the temperature to 55 degrees",
  "acknowledgement": "I will set the temperature to 55 degrees if you are experiencing snowing."
}

Input: switch off the lights if nobody is at home
Output: {
  "condition": "if nobody is at home",
  "action": "switch off the lights",
  "acknowledgement": "I will switch off the lights if you are not at home."
}

Input: lock the doors when I leave home
Output: {
  "condition": "the user leaves home",
  "action": "lock the doors",
  "acknowledgement": "I will make sure to lock the doors when you leave home."
}

Input: turn on the heater when it is below 50 degrees
Output: {
  "condition": "the temperature is below 50 degrees",
  "action": "turn on the heater",
  "acknowledgement": "I will turn on the heater when your home's temperature drops below 50 degrees."
}

Input: reduce the temperature by 5 degrees
Out

## Multiple Conditions

In [6]:
advanced_commands = [
    "When evening falls and the daylight fades, activate the warm ambient lighting, and close the blinds for a cozy, secure atmosphere.",
    "As the outdoor temperature drops below a comfortable level, adjust the thermostat to maintain warmth, and ignite the fireplace for added comfort.",
    "Upon the morning alarm sounding, open the curtains to welcome the day, and start brewing a fresh pot of coffee to ease into the morning.",
    "In the event of rain intensifying and no one's home, close any open windows to prevent water intrusion, and illuminate the indoor space for a brighter environment.",
    "When the doorbell signals a visitor's arrival, display the live video feed from the security camera, and if the guest is recognized, unlock the door while notifying me of their presence.",
]

Reuse the same prompt:

In [7]:
for advanced_command in advanced_commands:

    prompt = f"""
    You are a syntax parser for a home automation system. Given a user's sentence like:
    
    if <condition> then <action>
    
    perform the following:

    1. Extract <condition> and <action>. If no condition, set it to None.
    2. Rewrite the extracted <condition> and <action> so that any of the user's personal and possessive pronouns are replaced with "the user".
    3. Create a friendly acknowledgement text. Replace any possessive references to the user with the corresponding second person
       possessive pronoun. References to the home automation system are to be in the first person.
    

    Process this sentence using the above steps:

    {advanced_command}

    Return the result in this format:

    {{
     
      "condition": <the rewritten condition from step 2>
      "action": <the rewritten action from step 2>
      "acknowledgement": <the acknowledgement text from step 3>

    }}

    """

    completion = generate_completion(prompt)

    print(f"Input: {advanced_command}")
    print(f"Output: {completion}")
    print()

Input: When evening falls and the daylight fades, activate the warm ambient lighting, and close the blinds for a cozy, secure atmosphere.
Output: {
  "condition": "When evening falls and the daylight fades",
  "action": "activate the warm ambient lighting, and close the blinds for a cozy, secure atmosphere",
  "acknowledgement": "I will activate the warm ambient lighting and close the blinds for your cozy and secure atmosphere."
}

Input: As the outdoor temperature drops below a comfortable level, adjust the thermostat to maintain warmth, and ignite the fireplace for added comfort.
Output: {
  "condition": "the outdoor temperature drops below a comfortable level",
  "action": "adjust the thermostat to maintain warmth, and ignite the fireplace for added comfort",
  "acknowledgement": "I will make sure to adjust the thermostat and ignite the fireplace to keep you warm when the temperature drops below a comfortable level."
}

Input: Upon the morning alarm sounding, open the curtains to we

Tweak the prompt a bit:

In [8]:
for advanced_command in advanced_commands:

    prompt = f"""
    You are a syntax parser for a home automation system. Given a user's sentence like:
    
    if <condition> then <action>
    
    perform the following:

    1. Extract <condition> and <action>. If no condition, set it to None.
    2. Rewrite the extracted <condition> and <action> so that any of the user's personal and possessive pronouns are replaced with "the user".
    3. Create a friendly acknowledgement text. Replace any possessive references to the user with the corresponding second person
       possessive pronoun. References to the home automation system are to be in the first person.
    

    Process this sentence using the above steps:

    {advanced_command}

    Return the result in this format:

    {{
     
      "condition": <the rewritten conditions from step 2 as a list> 
      "action": <the rewritten actions from step 2> as a as a list
      "acknowledgement": <the acknowledgement text from step 3>

    }}

    """

    completion = generate_completion(prompt)

    print(f"Input: {advanced_command}")
    print(f"Output: {completion}")
    print()

Input: When evening falls and the daylight fades, activate the warm ambient lighting, and close the blinds for a cozy, secure atmosphere.
Output: {
     
  "condition": ["When evening falls", "the daylight fades"], 
  "action": ["activate the warm ambient lighting", "close the blinds for a cozy, secure atmosphere"],
  "acknowledgement": "I will activate the warm ambient lighting and close the blinds for your cozy and secure atmosphere."

}

Input: As the outdoor temperature drops below a comfortable level, adjust the thermostat to maintain warmth, and ignite the fireplace for added comfort.
Output: {
     
      "condition": ["the outdoor temperature drops below a comfortable level"],
      "action": ["adjust the thermostat to maintain warmth", "ignite the fireplace for added comfort"],
      "acknowledgement": "I will adjust the thermostat to maintain warmth and ignite the fireplace for your added comfort."
      
    }

Input: Upon the morning alarm sounding, open the curtains to wel